In [1]:
# libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
from keras.optimizers import SGD
import math
from sklearn.metrics import mean_squared_error
import quandl as q

plt.style.use('seaborn-darkgrid')


Using TensorFlow backend.


In [2]:
#https://medium.com/datadriveninvestor/multivariate-time-series-using-rnn-with-keras-7f78f4488679

In [3]:
#Import 3 assets in portfolio
asset_list = ['MCD','JPM','NVDA']

def GetData(asset_name):
    return pd.read_csv('Asset_Dataset/'+asset_name+'.csv', usecols=['Date','Adj Close'], parse_dates=True, index_col='Date' ).dropna()
    
    

In [4]:
data_MCD = GetData('MCD')
data_MCD.head()

,Adj Close
Date,
2014-06-02,87.501846
2014-06-09,86.240288
2014-06-16,87.467529
2014-06-23,87.072746
2014-06-30,86.660828


In [5]:
#import macro data
data_gdp = q.get("FRED/GDPC1", start_date="2014-04-01", end_date="2019-06-01",collapse='daily', 
                 authtoken="oWpRXksxc4gyrtAwXe18")
%store data_gdp

data_savings = q.get("FRED/PSAVERT", start_date="2014-04-01", end_date="2019-06-01",collapse='daily', 
                     authtoken="oWpRXksxc4gyrtAwXe18")
%store data_savings

#import market sentiment adjusted close price
data_vix = pd.read_csv('Asset_Dataset/VIX_daily.csv', header=0,usecols=['Date','Adj Close'], 
                   parse_dates=True, index_col='Date')
%store data_vix

Stored 'data_gdp' (DataFrame)
Stored 'data_savings' (DataFrame)
Stored 'data_vix' (DataFrame)


In [6]:
data_m = pd.concat([data_savings,data_vix],axis=1)

In [7]:
data_m = data_m.fillna(method='ffill')


In [8]:
data_all = pd.concat([data_MCD,data_m],axis=1).dropna()
data_all.columns = ['MCD_close','Savings','VIX']
data_all

,MCD_close,Savings,VIX
Date,,,
2014-06-02,87.501846,7.5,11.580000
2014-06-09,86.240288,7.5,11.150000
2014-06-16,87.467529,7.5,12.650000
2014-06-23,87.072746,7.5,10.980000
2014-06-30,86.660828,7.5,11.570000
2014-07-07,86.137337,7.5,11.330000
2014-07-14,84.952988,7.5,11.820000
2014-07-21,82.146683,7.5,12.810000
2014-07-28,80.928070,7.5,12.560000


In [9]:
#drop MCD 
data_m = data_all.drop('MCD_close', 1)

In [10]:
#scale input
scaler = MinMaxScaler(feature_range=(0,1))
scaled_input = scaler.fit_transform(data_m)
print(scaled_input)
%store scaled_input

[[0.48       0.07133758]
 [0.48       0.05764331]
 [0.48       0.10541401]
 [0.48       0.0522293 ]
 [0.48       0.0710191 ]
 [0.48       0.06337579]
 [0.48       0.07898089]
 [0.48       0.11050955]
 [0.48       0.10254776]
 [0.4        0.18407642]
 [0.4        0.15573247]
 [0.4        0.09490445]
 [0.4        0.07515923]
 [0.48       0.08757961]
 [0.48       0.10573248]
 [0.48       0.15222929]
 [0.48       0.13853502]
 [0.48       0.21146495]
 [0.4        0.19490445]
 [0.4        0.48726108]
 [0.4        0.29394903]
 [0.4        0.21337581]
 [0.44       0.17165604]
 [0.44       0.10605095]
 [0.44       0.14808916]
 [0.44       0.10445859]
 [0.56       0.15350317]
 [0.56       0.15509553]
 [0.56       0.35286622]
 [0.56       0.18821655]
 [0.56       0.18216559]
 [0.6        0.33694265]
 [0.6        0.32675157]
 [0.6        0.19681527]
 [0.64       0.32133756]
 [0.64       0.29331205]
 [0.64       0.16624203]
 [0.44       0.11783439]
 [0.44       0.18216559]
 [0.44       0.19968152]


In [36]:
lookback= 2

test_size=int(.2 * len(data_MCD))
X=[]
y=[]
for i in range(len(data_MCD)-lookback):
    t=[]
    for j in range(0,lookback):
        
        t.append(scaled_input[[(i+j)], :])
    X.append(t)
    y.append(scaled_input[i+ lookback,1])

IndexError: index 238 is out of bounds for axis 0 with size 238

In [30]:
X, y= np.array(X), np.array(y)
X_test = X[:test_size+lookback]
X = X.reshape(X.shape[0],lookback, 2)-1
X_test = X_test.reshape(X_test.shape[0],lookback, 2)
print(X.shape)
print(X_test.shape)

(237, 2, 2)
(54, 2, 2)


In [31]:
model = Sequential()
model.add(LSTM(units=30, return_sequences= True, input_shape=(X.shape[1],2)))
model.add(LSTM(units=30, return_sequences=True))
model.add(LSTM(units=30))
model.add(Dense(units=1))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_13 (LSTM)               (None, 2, 30)             3960      
_________________________________________________________________
lstm_14 (LSTM)               (None, 2, 30)             7320      
_________________________________________________________________
lstm_15 (LSTM)               (None, 30)                7320      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 31        
Total params: 18,631
Trainable params: 18,631
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.compile(optimizer='adam', loss='mean_squared_error')


In [34]:
model.fit(X, y, epochs=200, batch_size=32)

ValueError: Input arrays should have the same number of samples as target arrays. Found 237 input samples and 236 target samples.